In [1]:
# After dashboard3.py (since demographics data was a failure, trying crime data)
# Using OpenData API: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i
# GOAL: downloading crime data using OPEN NYC api. Getting census tract using Census API, outputting to csv. building0_correlation_API_DATA3_ToGeoidCrimeCount will get the file and make it a correlation sv to be used
import pandas as pd
import censusdata
import numpy as np
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
import statsmodels.formula.api as sm

from scipy import stats
import plotly.express as px         # plotting

from sodapy import Socrata             # NYC Open Data

from urllib.request import urlopen     # Turnign long an dlat to census geoids                                                  # Getting the json data from the url
import requests

In [2]:
crimeOutFile = "DataFiles/Crime and Demographics/Crime/ACS_10_5YR_S0101_With_Census_Data2.csv"

USERNAME = "mhasan0047@gmail.com"
PASSWORD = "s0mePass212"
APP_TOKEN = "I2iI5wKFvjnajBahyxLb5IjBz"
MyAppToken = "I2iI5wKFvjnajBahyxLb5IjBz"

queryLimit = 1000    # The amount of crime reports to download from nycopen api
fromRow    = 0       # Speicy ange of reports to focus on and find geoid of using census api (Takes long time)
toRow      = 100

In [3]:
# 1) Gettign Crime data from OpenData
print("...Downloading Crime data...")
client = Socrata("data.cityofnewyork.us",
    APP_TOKEN,
    username = USERNAME,
    password = PASSWORD, 
    timeout = 500
)
results = client.get(
    "qgea-i56i", 
    # query = "SELECT * WHERE boro_nm = 'MANHATTAN' OR boro_nm = 'QUEENS' OR boro_nm = 'BROOKLYN' OR boro_nm = 'BRONX' OR boro_nm = 'QUEENS'"
    limit = queryLimit
)
crimeDF = pd.DataFrame.from_records(results)
crimeDF

...Downloading Crime data...


,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,pd_desc,crm_atpt_cptd_cd,...,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,cmplnt_to_dt,cmplnt_to_tm,susp_age_group,hadevelopt,transit_district,station_name
0,522575447,2006-08-29T00:00:00.000,13:00:00,43,2006-08-30T00:00:00.000,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,...,58,5,31,26,NaN,NaN,NaN,NaN,NaN,NaN
1,403507361,2006-11-05T00:00:00.000,11:00:00,66,2006-11-05T00:00:00.000,107,BURGLARY,221,"BURGLARY,RESIDENCE,DAY",COMPLETED,...,2,2,7,39,2006-11-05T00:00:00.000,17:40:00,NaN,NaN,NaN,NaN
2,631420068,2006-09-08T00:00:00.000,23:30:00,106,2006-09-09T00:00:00.000,347,INTOXICATED & IMPAIRED DRIVING,905,"INTOXICATED DRIVING,ALCOHOL",COMPLETED,...,62,3,41,64,2006-09-09T00:00:00.000,00:01:00,NaN,NaN,NaN,NaN
3,995609899,2011-12-13T00:00:00.000,18:40:00,79,2011-12-13T00:00:00.000,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,...,69,2,49,51,2011-12-13T00:00:00.000,18:49:00,NaN,NaN,NaN,NaN
4,480667624,2009-08-14T00:00:00.000,04:20:00,30,2009-08-14T00:00:00.000,113,FORGERY,729,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,...,47,4,23,21,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,270470854,2006-05-20T00:00:00.000,19:45:00,75,2006-05-20T00:00:00.000,235,DANGEROUS DRUGS,511,"CONTROLLED SUBSTANCE, POSSESSI",COMPLETED,...,45,2,25,47,NaN,NaN,NaN,NaN,NaN,NaN
199996,628157213,2007-05-07T00:00:00.000,19:00:00,44,2007-05-07T00:00:00.000,341,PETIT LARCENY,321,"LARCENY,PETIT FROM AUTO",COMPLETED,...,50,5,35,27,2007-05-07T00:00:00.000,22:30:00,NaN,NaN,NaN,NaN
199997,451193274,2006-05-21T00:00:00.000,18:00:00,107,2006-05-28T00:00:00.000,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",COMPLETED,...,25,3,24,65,2006-05-28T00:00:00.000,13:00:00,NaN,POMONOK,NaN,NaN
199998,961084522,2009-01-22T00:00:00.000,05:00:00,73,2009-01-26T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,...,55,2,17,46,NaN,NaN,<18,NaN,NaN,NaN


In [4]:
# 2) Appending New Cols - Cols for census data:
crimeDF["Geoid"] = int
crimeDF["Date"]  = str         #rpt_dt
crimeDF["Month"] = int
crimeDF["Year"]  = int
crimeDF["CrimeComplaint"] = str

expandCols = [ "CensusTract_2010", "CensusBlock_2010", "CountyName", "Geoid_FULL", 
    "CensusTract_2010_ID", "CensusTract_2010_NAME", "CensusBlock_2010_ID", "CensusBlock_2010_NAME"]  
currCols = list(crimeDF.columns)
for newCol in expandCols:
    if newCol not in currCols:
        crimeDF[newCol] = np.str
crimeDF

,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,pd_desc,crm_atpt_cptd_cd,...,Year,CrimeComplaint,CensusTract_2010,CensusBlock_2010,CountyName,Geoid_FULL,CensusTract_2010_ID,CensusTract_2010_NAME,CensusBlock_2010_ID,CensusBlock_2010_NAME
0,522575447,2006-08-29T00:00:00.000,13:00:00,43,2006-08-30T00:00:00.000,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,403507361,2006-11-05T00:00:00.000,11:00:00,66,2006-11-05T00:00:00.000,107,BURGLARY,221,"BURGLARY,RESIDENCE,DAY",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,631420068,2006-09-08T00:00:00.000,23:30:00,106,2006-09-09T00:00:00.000,347,INTOXICATED & IMPAIRED DRIVING,905,"INTOXICATED DRIVING,ALCOHOL",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,995609899,2011-12-13T00:00:00.000,18:40:00,79,2011-12-13T00:00:00.000,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,480667624,2009-08-14T00:00:00.000,04:20:00,30,2009-08-14T00:00:00.000,113,FORGERY,729,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,270470854,2006-05-20T00:00:00.000,19:45:00,75,2006-05-20T00:00:00.000,235,DANGEROUS DRUGS,511,"CONTROLLED SUBSTANCE, POSSESSI",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
199996,628157213,2007-05-07T00:00:00.000,19:00:00,44,2007-05-07T00:00:00.000,341,PETIT LARCENY,321,"LARCENY,PETIT FROM AUTO",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
199997,451193274,2006-05-21T00:00:00.000,18:00:00,107,2006-05-28T00:00:00.000,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
199998,961084522,2009-01-22T00:00:00.000,05:00:00,73,2009-01-26T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,...,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [5]:
def getCensusTract(longitude, latitude,retryRun=0):                                                                 # returns an array [censusTract, CensusBlock, CountyName]
    #url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={0}&y={1}&benchmark=Public_AR_Current&vintage=Current_Current&format=json".format(longitude,latitude)
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={0}&y={1}&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&format=json".format(longitude,latitude)
    if retryRun == 11:                                                                                              # Failed to get json data 11 times with this longitude and latitude so need to skip this one
        print("*****Failed 11 times to get geodata so will insert 'error'*****")
        return [str("error"), str("error"), str("error")]
    try:
        response = requests.get(url)
        dataJSON = response.json()
        data    = dataJSON["result"]

        tractNAME     = data["geographies"]["Census Tracts"][0]["NAME"]
        tractBASENAME = data["geographies"]["Census Tracts"][0]["BASENAME"]
        tractID       = data["geographies"]["Census Tracts"][0]["TRACT"]
        countyNAME    = data["geographies"]["Counties"][0]["NAME"] 
        blockGEOID =  data["geographies"]["Census Blocks"][0]["GEOID"]
        blockNAME     = data["geographies"]["Census Blocks"][0]["NAME"] 
        blockBASENAME = data["geographies"]["Census Blocks"][0]["BASENAME"]
        blockID       = data["geographies"]["Census Blocks"][0]["BLOCK"]
        # Returns: tractBASENAME, blockBASENAME, countyName, geoid, tractid and name, block id and name
        return [
            str(tractBASENAME), str(blockBASENAME), str(countyNAME), str(blockGEOID), 
            str(tractID), str(tractNAME), str(blockID), str(blockNAME)
        ]
    except:
        retryRun+=1
        print("_____Error on longitude, latitude: "+str(longitude)+","+str(latitude) + ".....retrying... "+str(retryRun))
        return getCensusTract(longitude, latitude,retryRun)                                                         # need to return the recursive function
    return

In [ ]:
# 3) Using Long and Lat to populate the Census Data Cols
crimeDFALL     = crimeDF.copy()
crimeDF        = crimeDF[fromRow:toRow]

print("...Getting Census Information of Crime Reports from Longitude and Latitude Coordinates...\n...Should take some a long time...")
rowsToDel = []
for row in range(0,len(crimeDF)):
    retryRun = 0
    lat = float(crimeDF.iloc[row]["latitude"])
    lon = float(crimeDF.iloc[row]["longitude"])
    if np.isnan(lon) or np.isnan(lat):
        print("***This report has no Longitude and Latitude Data so will skip it and delete it from the df***")
        rowsToDel.append(row)
        continue
    
    returnArray = getCensusTract(lon, lat)            # returnArray = [tractBASENAME, blockBASENAME, countyName, geoid, tract id, tract name, block id, block name]
    # Make sure the "expandCols" index and "returnArray" index are the same so it prints to right cols
    if len(expandCols) == len(returnArray):
        for colToWrite in range(0, len(expandCols)):
            crimeDF.at[row, expandCols[colToWrite]] = returnArray[colToWrite] 
    else:
        print("*** Number of col to add and values to polulate cols are not the same!! ******") 
    if row%10 == 0:
        print(row)
        crimeDF.to_csv(crimeOutFile, index=False)   # Saving just in case 
crimeDF

...Getting Census Information of Crime Reports from Longitude and Latitude Coordinates...
...Should take some a long time...
0
10
***This report has no Longitude and Latitude Data so will skip it and delete it from the df***
20
30
40
50
***This report has no Longitude and Latitude Data so will skip it and delete it from the df***
60
70
80
90
100


In [ ]:
# 4) Delete rows of crime reports that had no long and lat data:
for rowToDel in rowsToDel:
    crimeDF = crimeDF.drop([rowToDel])  
    
crimeDF = crimeDF.reset_index(drop=True)   
crimeDF.to_csv(crimeOutFile, index=False)
crimeDF

In [ ]:
# 5) Populate the Geoid (exclude block id from, Date, Month, Year, and CrimeComplaint Cols
for row in range(0,len(crimeDF)):
    crimeDF.at[row, "Geoid"] = int(str(crimeDF.iloc[row]["Geoid_FULL"])[0:11])
    crimeDF.at[row, "CrimeComplaint"] = "["+crimeDF.iloc[row]["ofns_desc"]+"] " + crimeDF.iloc[row]["pd_desc"] 
    date = crimeDF.iloc[row]["rpt_dt"].split("T")[0].split("-")
    dateMDY = date[2]+"-"+date[1]+"-"+date[0]
    crimeDF.at[row, "Date"] = dateMDY
    crimeDF.at[row, "Year"] = int(date[0])
    crimeDF.at[row, "Month"] = int(date[1])
crimeDF = crimeDF.sort_values(by=['Year', 'Month', 'ofns_desc', 'CrimeComplaint', 'CountyName','Geoid'], ascending=False)
crimeDF = crimeDF.reset_index(drop=True)   
crimeDF.to_csv(crimeOutFile, index=False)
crimeDF
